## In this notebook we will be using YOLOv8 model from ultralytics to train bone-fracture dataset can be downloaded from Roboflow 100 universe - [here](https://universe.roboflow.com/roboflow-100/bone-fracture-7fylg).


#### If you got any suggestions for code improvement or questions. Please reach out.

# Imports

In [1]:
import os
import json
import glob
import copy
from tqdm import tqdm
from pathlib import Path
import yaml
import shutil

import cv2
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pybboxes as pbx

import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
from torchvision.ops import box_convert
from torchvision.utils import draw_bounding_boxes
from torchvision.transforms import ToPILImage

#### Give path to the image directory

In [2]:
train_dir = 'train/'
val_dir = 'val/'
test_dir = 'test/'

#### In all the three split data directories you will find json file, which contains information about the image_id, filenames, height, width and other metadata like bboxes, area, category_id etc.

#### The make_df function will help us create csv files for the three splits. These will make it easy for us to manipulate the data. Here, we have used list comprehension to make the code look clean and efficient. 

In [3]:
import json
import pandas as pd

def make_df(modes):
    for mode in modes:
        with open(f'{mode}/_annotations.coco.json', 'r') as f:
            data = json.load(f)

        # Extract information from JSON data
        image_ids = [image['id'] for image in data['images']]
        file_names = [image['file_name'] for image in data['images']]
        heights = [image['height'] for image in data['images']]
        widths = [image['width'] for image in data['images']]

        annotation_ids = [annot['id'] for annot in data['annotations']]
        image_ids_annot = [annot['image_id'] for annot in data['annotations']]
        category_ids = [annot['category_id'] for annot in data['annotations']]
        bboxes = [annot['bbox'] for annot in data['annotations']]
        areas = [annot['area'] for annot in data['annotations']]

        # Create pandas dataframe
        df = pd.DataFrame({
            'image_id': image_ids_annot,
            'file_name': [file_names[id] for id in image_ids_annot],
            'height': [heights[id] for id in image_ids_annot],
            'width': [widths[id] for id in image_ids_annot],
            'category_id': category_ids,
            'bbox': bboxes,
            'area': areas,
        })

        # Add category names to dataframe
        category_names = {category['id']: category['name'] for category in data['categories']}
        df['category_name'] = df['category_id'].map(category_names)

        # Save dataframe to CSV file
        df.to_csv(f'{mode}_data.csv', index=False)

In [4]:
modes =['train', 'val', 'test']
make_df(modes)

In [5]:
train = pd.read_csv('train_data.csv')
valid = pd.read_csv('valid_data.csv')
test = pd.read_csv('test_data.csv')

In [6]:
train.head()

,image_id,file_name,height,width,category_id,bbox,area,category_name
0,0,70_jpg.rf.a45b1f9b3f335f0cc7210b9dc2f858cf.jpg,1024,752,4,"[242, 211, 125, 175]",21875,messed_up_angle
1,0,70_jpg.rf.a45b1f9b3f335f0cc7210b9dc2f858cf.jpg,1024,752,4,"[297, 502, 144, 147]",21168,messed_up_angle
2,1,29_jpg.rf.a358d0249bf4ce4ecc85be891f7d1721.jpg,1024,448,2,"[243, 618, 27, 27]",729,fracture
3,2,47_jpg.rf.a7792e1d649b43ecabeb2507a006e9ba.jpg,420,348,2,"[241, 271, 21, 12]",252,fracture
4,3,166_jpg.rf.aa2a38ccf92bcff3752e3fb5f5fe42ed.jpg,360,360,2,"[125, 111, 77, 108]",8316,fracture


#### I used a different code to convert the bbox from string format to float numbers and then assigning them to the corresponding x, y, w, h columns made in the dataframe. But this code is short and readable. copied it from kaggle.

In [126]:
def preprocess_bbox(df):
    bboxs = np.stack(df['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
    for i, column in enumerate(['x', 'y', 'w', 'h']):
        df[column] = bboxs[:,i]
    df.drop(columns=['bbox'], inplace=True)
    return df

In [127]:
train_df = preprocess_bbox(train)
val_df = preprocess_bbox(valid)

#### Now to convert the csv file to yaml file, which is needed for yolo.  Let's first create a directory to store data. we have to be careful here because yolo reads data from the yaml file, which provides the path to the images and it's labels. Images and labels have to be of the same name. 

In [130]:
# make output_dir
dest_dir = "YOLO_DATA"
!mkdir {dest_dir}

_ = Path(f"{dest_dir}/dataset.yaml").write_text(f"""path: {dest_dir}
train: D:\\CODE\\Bone fracture\\YOLO_DATA\\images\\train
val: D:\\CODE\\Bone fracture\\YOLO_DATA\\images\\val


nc: 4
names: ['fracture', 'line', 'messed_up_angle', 'angle']
""")

#### create_txt_file function converts the [x, y, w, h] coco format bboxes to yolo format [xcenter, ycenter, width, height]. 
#### Note- width and height in yolo format is for the bounding box and all the four values should be normalized between 0 and 1.

In [137]:
def create_txt_file(path: Path, bboxes, width, height):
    """Creates a .txt file with annotation strings for the given bounding boxes"""
    
    anno_str = []
    for bbox in bboxes:
        x, y, w, h = bbox[0], bbox[1], bbox[2], bbox[3]
        xc = x + w / 2
        yc = y + h / 2
        xc /= width
        yc /= height
        w /= width
        h /= height
        anno_str.append(f"0 {xc} {yc} {w} {h}")
    path.write_text("\n".join(anno_str))

#### The below code organizes image and label files into separate directories based on a dataFrame. It iterates over two modes, "train" and "val", and stores them to their respected mode directories for storing images and labels. It then retrieves relevant information from the dataFrame, such as file names, widths, heights, and bounding box coordinates. The code copies image files to the appropriate directory and creates label files in a specific format. Lastly, it stores the paths of the image and label files in a list called path_list. I found this code to be very efficient and elegant. you can easily modify it for your use case.

In [138]:
path_list = []
for mode in ["train", "val"]:
    image_folder = Path(dest_dir) / "images" / f"{mode}"
    image_folder.mkdir(parents=True, exist_ok=True)

    label_folder = Path(dest_dir) / "labels" / f"{mode}"
    label_folder.mkdir(parents=True, exist_ok=True)

    df = locals().get(f"{mode}_df")

    grouped = df.groupby('file_name')
    for image_id, group_df in tqdm(grouped, total=len(grouped)):
        file_name = group_df.iloc[0].file_name
        width, height = group_df.iloc[0].width, group_df.iloc[0].height
        bboxes = [(row.x, row.y, row.w, row.h) for _, row in group_df.iterrows()]
        img_path = image_folder / f"{file_name}.jpg"
        label_path = label_folder / f"{file_name}.txt"
        shutil.copy(f"{mode}/{file_name}", img_path)
        create_txt_file(label_path, bboxes, width, height)
        path_list.append((str(img_path), str(label_path)))

100%|█████████████████████████████████████████████████████████████████████████████████| 83/83 [00:00<00:00, 530.57it/s]


In [133]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [142]:
import ultralytics
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8n.pt")
model.to(device)

#### Careful here while providing the path to the data. 

In [184]:
# Training.

results = model.train(
   data="D:\\CODE\\Bone fracture\\YOLO_DATA\\dataset.yaml",
   imgsz=720,
   epochs=20,
   batch=8,
    workers=2,
    device='0',
   name='yolov8n_v1_1epoch'
)

New https://pypi.org/project/ultralytics/8.0.86 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.66  Python-3.11.0 torch-2.0.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
yolo\engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=D:\CODE\Bone fracture\YOLO_DATA\dataset.yaml, epochs=20, patience=50, batch=8, imgsz=720, save=True, save_period=-1, cache=False, device=0, workers=2, project=None, name=yolov8n_v1_1epoch, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_thickness=3, visualize=False, augmen

In [185]:
result_paths = glob.glob('runs\detect\yolov8n_v1_1epoch2/*jpg') +  glob.glob('runs\detect\yolov8n_v1_1epoch2/*png')

In [ ]:
%matplotlib inline

for path in result_paths:
    image = cv2.imread(path)
    plt.figure(figsize=(14,10))
    plt.imshow(image)
    plt.show()

In [186]:
model_best = YOLO("runs\detect\yolov8n_v1_1epoch2\weights\\best.pt")

In [187]:
preds = model_best.predict(conf = 0.1, source = "D:\\CODE\\Bone fracture\\test")


image 1/44 D:\CODE\Bone fracture\test\105_jpg.rf.3cde2fcd15a9bdf6a2d2d32aff48f33d.jpg: 736x448 3 fractures, 43.1ms
image 2/44 D:\CODE\Bone fracture\test\10_jpg.rf.d362a00f9a6b4ac31668dc8aae9c71de.jpg: 736x736 8 fractures, 75.5ms
image 3/44 D:\CODE\Bone fracture\test\117_jpg.rf.119dccd2483b04d8d3a8c33a1393d362.jpg: 736x256 4 fractures, 22.6ms
image 4/44 D:\CODE\Bone fracture\test\118_jpg.rf.acee2a86eba65adc57f3b15d5acab93c.jpg: 736x288 (no detections), 20.9ms
image 5/44 D:\CODE\Bone fracture\test\11_jpg.rf.8e1c22ba2779121f3ba0a8ae03a20407.jpg: 736x448 1 fracture, 10.1ms
image 6/44 D:\CODE\Bone fracture\test\124_jpg.rf.100aeaede7a9c017d7f74f73cfcf34d7.jpg: 736x608 (no detections), 10.5ms
image 7/44 D:\CODE\Bone fracture\test\124_jpg.rf.dab4a5f6292af8332da8f9bad9751ba6.jpg: 736x256 1 fracture, 9.6ms
image 8/44 D:\CODE\Bone fracture\test\12_jpg.rf.3e4b2f52016c100e9f869397933ef2d3.jpg: 736x224 (no detections), 10.1ms
image 9/44 D:\CODE\Bone fracture\test\131_jpg.rf.1a70e4c9e91ea953aa55988a

In [188]:
test_images_sort = os.listdir(test_dir)
#test_images_sort

In [195]:
test_preds = pd.DataFrame(columns=range(6))
for i in range(len(preds)):
    arri = pd.DataFrame(preds[i].boxes.boxes.cpu()).astype(float)
    path = test_images_sort[i]
    file = path.split('/')[-1]
    arri = arri.assign(file=file)
    arri = arri.assign(i=i)
    test_preds = pd.concat([test_preds,arri],axis=0)
test_preds.columns = ['x','y','w','h','confidence','class','file_name','i']
display(test_preds)

,x,y,w,h,confidence,class,file_name,i
0,232.354324,637.304871,333.971039,689.682983,0.200192,0.0,105_jpg.rf.3cde2fcd15a9bdf6a2d2d32aff48f33d.jpg,0.0
1,206.505142,607.038513,341.276001,687.368530,0.196243,0.0,105_jpg.rf.3cde2fcd15a9bdf6a2d2d32aff48f33d.jpg,0.0
2,259.868469,639.646118,331.049011,688.943665,0.152937,0.0,105_jpg.rf.3cde2fcd15a9bdf6a2d2d32aff48f33d.jpg,0.0
0,495.719452,589.294250,546.936829,624.827698,0.259377,0.0,10_jpg.rf.d362a00f9a6b4ac31668dc8aae9c71de.jpg,1.0
1,494.625183,554.118103,623.516663,630.272034,0.222910,0.0,10_jpg.rf.d362a00f9a6b4ac31668dc8aae9c71de.jpg,1.0
...,...,...,...,...,...,...,...,...
1,529.039185,486.478180,563.995667,524.549683,0.143871,0.0,7_jpg.rf.5b79fa048d8e493e13436fdae01bae0c.jpg,41.0
2,441.089935,399.950317,512.693970,465.411102,0.101195,0.0,7_jpg.rf.5b79fa048d8e493e13436fdae01bae0c.jpg,41.0
0,84.165855,163.834335,108.210091,190.455505,0.118584,0.0,90_jpg.rf.7163e0a2586c369f8cb72f1cdc305d52.jpg,42.0
1,83.596741,159.201080,112.293900,191.270416,0.117746,0.0,90_jpg.rf.7163e0a2586c369f8cb72f1cdc305d52.jpg,42.0


In [196]:
test_preds = test_preds[["file_name","x","y","w","h","confidence"]].reset_index(drop=True)

In [197]:
test_preds

,file_name,x,y,w,h,confidence
0,105_jpg.rf.3cde2fcd15a9bdf6a2d2d32aff48f33d.jpg,232.354324,637.304871,333.971039,689.682983,0.200192
1,105_jpg.rf.3cde2fcd15a9bdf6a2d2d32aff48f33d.jpg,206.505142,607.038513,341.276001,687.368530,0.196243
2,105_jpg.rf.3cde2fcd15a9bdf6a2d2d32aff48f33d.jpg,259.868469,639.646118,331.049011,688.943665,0.152937
3,10_jpg.rf.d362a00f9a6b4ac31668dc8aae9c71de.jpg,495.719452,589.294250,546.936829,624.827698,0.259377
4,10_jpg.rf.d362a00f9a6b4ac31668dc8aae9c71de.jpg,494.625183,554.118103,623.516663,630.272034,0.222910
...,...,...,...,...,...,...
91,7_jpg.rf.5b79fa048d8e493e13436fdae01bae0c.jpg,529.039185,486.478180,563.995667,524.549683,0.143871
92,7_jpg.rf.5b79fa048d8e493e13436fdae01bae0c.jpg,441.089935,399.950317,512.693970,465.411102,0.101195
93,90_jpg.rf.7163e0a2586c369f8cb72f1cdc305d52.jpg,84.165855,163.834335,108.210091,190.455505,0.118584
94,90_jpg.rf.7163e0a2586c369f8cb72f1cdc305d52.jpg,83.596741,159.201080,112.293900,191.270416,0.117746


In [ ]:
def display_img(img_path):

    image = os.path.join(f'{test_dir}/{img_path}')
    img = cv2.imread(image)

    fig, ax = plt.subplots(figsize=(15, 15))
    ax.imshow(img)
    
    rows = test_preds[test_preds['file_name'] == img_path]
    bboxes = []
    for _,row in rows.iterrows():
        bboxes.append((row.x, row.y, row.w, row.h))
    for bbox in bboxes:
        rect = patches.Rectangle((bbox[0],bbox[1]),bbox[2],bbox[3],linewidth=1,edgecolor='r',facecolor='none')
        ax.add_patch(rect)

    plt.show()

display_img('90_jpg.rf.7163e0a2586c369f8cb72f1cdc305d52.jpg')